# PyTorch: nn

연산 그래프와 autograd는 복잡한 연산자를 정의하고 도함수(derivative)를 자동으로 계산하는데 매우 강력한 패러다임(paradigm)입니다. 하지만 규모가 큰 신경망에서는 autograd 그 자체만으로는 너무 낮은 수준(low-level)일 수 있습니다.

신경망을 구성할 때 종종 연산을 여러 계층으로 배열하는 것으로 생각하게 되는데, 이 중 일부는 학습 도중 최적화가 될 학습 가능한 매개변수를 갖고 있습니다.

Tensorflow에서 Keras, TensorFlow-Slim,이나 TFLearn 같은 패키지는 원초적(raw)인 연산 그래프보다 더 높은 수준의 추상화(higher-level abstraction)를 제공하여 신경망을 구축하는데 유용합니다.

PyTorch에서는 **nn** 패키지가 동일한 목적으로 제공됩니다. **nn** 패키지는 대략 신경망 계층들과 동일한 모듈의 집합을 정의합니다. 모듈은 입력 Tensor를 받고 출력 Tensor를 계산하는 한편, 학습 가능한 매개변수를 포함하는 Tensor와 같은 내부 상태(internal state)를 갖습니다. 또한 **nn** 패키지는 신경망을 학습시킬 때 주로 사용하는 유용한 손실 함수들도 정의합니다.

이번 예제에서는 **nn** 패키지를 사용하여 2-계층 신경망을 구성해보겠습니다.

In [1]:
import torch

In [2]:
# N은 배치 크기이며, D_in은 입력의 차원입니다;
# H는 은닉 계층의 차원이며, D_out은 출력 차원입니다.
N, D_in, H, D_out = 64, 1000, 100, 10

In [3]:
# 입력과 출력을 저장하기 위해 무작위 값을 갖는 Tensor를 생성합니다.
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

In [4]:
# nn 패키지를 사용하여 모델을 순차적인 계층(Sequence of layers)으로 정의합니다.
# nn.Sequential은 다른 모듈들을 포함하는 모듈로, 그 모듈들을 순차적으로 적용하여
# 출력을 생성합니다. 각각의 선형(Linear) 모듈은 선형 함수를 사용하여 입력으로부터
# 출력을 계산하고, 가중치와 편향(Bias)을 저장하기 위해 내부적인 Tensor를 갖습니다.
model = torch.nn.Sequential(
    torch.nn.Linear(D_in,H),
    torch.nn.ReLU(),
    torch.nn.Linear(H,D_out))

In [5]:
# 또한, nn 패키지에는 널리 사용하는 손실 함수들에 대한 정의도 포함하고 있습니다
# 여기에서는 평균 제곱 오차(MSE; Mean Squared Error)를 손실 함수로 사용하겠습니다.
loss_fn = torch.nn.MSELoss(size_average=False)

C:\ProgramData\Anaconda3\envs\keras_gpu\lib\site-packages\torch\nn\functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [6]:
learning_rate = 1e-4

In [8]:
for t in range(501):
    # 순전파 단계: 모델에 x를 전달하여 예상하는 y값을 계산합니다. 모듈 객체는
    # __call__ 연산자를 덮어써서(Override) 함수처럼 호출할 수 있게 합니다.
    # 그렇게 함으로써 입력 데이터의 Tensor를 모듈에 전달하고 출력 데이터의
    # Tensor를 생성합니다.
    y_pred = model(x)
    
    # 손실을 계산하고 출력합니다. 예측한 y값과 정답 y를 갖는 Tensor들을 전달하고,
    # 손실 함수는 손실(loss)을 갖는 Tensor를 반환합니다.
    loss = loss_fn(y_pred, y)
    if t%20 == 0:
        print(t, loss.item())
    
    # 역전파 단계를 실행하기 전에 변화도를 0으로 만듭니다.
    model.zero_grad()
    
    # 역전파 단계: 모델의 학습 가능한 모든 매개변수에 대해서 손실의 변화도를
    # 계산합니다. 내부적으로 각 모듈의 매개변수는 requires_grad=True 일 때
    # Tensor 내에 저장되므로, 이 호출은 모든 모델의 모든 학습 가능한 매개변수의
    # 변화도를 계산하게 됩니다.
    loss.backward()
    
    # 경사하강법(Gradient Descent)을 사용하여 가중치를 갱신합니다. 각 매개변수는
    # Tensor이므로 이전에 했던 것과 같이 변화도에 접근할 수 있습니다.
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

0 737.5242309570312
20 238.66172790527344
40 74.30953979492188
60 22.62376594543457
80 7.5960187911987305
100 2.833183765411377
120 1.1396064758300781
140 0.486367791891098
160 0.21742595732212067
180 0.10087085515260696
200 0.04827982932329178
220 0.023711487650871277
240 0.01189431268721819
260 0.006117545999586582
280 0.0032040865626186132
300 0.001703078392893076
320 0.0009163381764665246
340 0.0004981345846317708
360 0.00027312771999277174
380 0.0001508999412180856
400 8.392562449444085e-05
420 4.6949691750342026e-05
440 2.6403533411212265e-05
460 1.491861530666938e-05
480 8.46444527269341e-06
500 4.824560619454132e-06


# PyTorch: optim

지금까지는 (autograd에서 추적 기록을 피하기 위해 **torch.no_grad()** 또는 **.data**를 사용하여) 학습 가능한 매개변수를 갖는 Tensor를 직접 조작하며 모델의 가중치를 갱신하였습니다. 이것은 확률적 경사 하강법과 같은 간단한 최적화 알고리즘에서는 크게 부담이 되지는 않지만, 실제로 신경망을 학습할 때는 주로 AdaGrad, RMSProp, Adam 등과 같은 좀 더 정교한 Optimizer를 사용합니다.

PyTorch의 **optim** 패키지는 최적화 알고리즘의 아이디어를 추상화하고 일반적으로 사용하는 최적화 알고리즘의 구현체(implementation)를 제공합니다.

이 예제에서는 앞에서와 같이 nn 패키지를 사용하여 모델을 정의하지만, optim 패키지가 제공하는 Adam 알고리즘을 이용하여 모델을 최적화하겠습니다.

In [9]:
# -*- coding: utf-8 -*-
import torch

# N은 배치 크기이며, D_in은 입력의 차원입니다;
# H는 은닉 계층의 차원이며, D_out은 출력 차원입니다.
N, D_in, H, D_out = 64, 1000, 100, 10

# 입력과 출력을 저장하기 위해 무작위 값을 갖는 Tensor를 생성합니다.
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# nn 패키지를 사용하여 모델과 손실 함수를 정의합니다.
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)
loss_fn = torch.nn.MSELoss(size_average=False)

C:\ProgramData\Anaconda3\envs\keras_gpu\lib\site-packages\torch\nn\functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [10]:
# optim 패키지를 사용하여 모델의 가중치를 Optimizer를 정의합니다.
# 여기서는 Adam을 사용하겠습니다. optim 패키지는 다른 다양한 최적화 알고리즘을
# 포함하고 있습니다. Adam 생성자의 첫 번째 인자는 갱신해야 하는 Tensor를 
# Optimizer에 알려줍니다.
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
for t in range(501):
    # 순전파 단계: 모델에 x를 전달하여 예상하는 y 값을 계산합니다.
    y_pred = model(x)

    # 손실을 계산하고 출력합니다.
    loss = loss_fn(y_pred, y)
    if t%20 == 0:
        print(t, loss.item())
        
    # 역전파 단계 전에, Optimizer 객체를 사용하여(모델의 학습 가능한 가중치인)
    # 갱신할 Variable들에 대한 모든 변화도를 0으로 만듭니다. 이는 기본적으로,
    # .backward()를 호출할 때마다 변화도가 버퍼(Buffer)에 (덮어쓰지 않고) 누적되기
    # 때문입니다. 더 자세한 내용은 torch.autograd.backward에 대한 문서를 참조하세요.
    optimizer.zero_grad()
    
    # 역전파 단계: 모델의 배개변수에 대한 손실의 변화도를 계산합니다.
    loss.backward()
    
    # Optimizer의 step 함수를 호출하면 매개변수가 갱신됩니다.
    optimizer.step()

0 683.0213623046875
20 415.2614440917969
40 263.68072509765625
60 166.30349731445312
80 100.57727813720703
100 57.4378662109375
120 30.525074005126953
140 15.314674377441406
160 7.451175212860107
180 3.5562334060668945
200 1.7192933559417725
220 0.8719439506530762
240 0.4693046510219574
260 0.26596105098724365
280 0.15441344678401947
300 0.0893702432513237
320 0.050653453916311264
340 0.027783185243606567
360 0.014664623886346817
380 0.007435380946844816
400 0.0036199786700308323
420 0.001693956321105361
440 0.0007625159923918545
460 0.00033059692941606045
480 0.00013820109597872943
500 5.5742737458785996e-05


# PyTorch: 사용자 정의 nn 모듈

가끔은 기존 모듈의 순차적 구성보다 더 복잡한 모델을 구성해야 할 때가 있습니다. 이럴 때는 nn.Module의 서브클래스로 새 모듈을 정의하고, 입력 Tensor를 받아 다른 모듈 또는 Tensor의 autograd 연산을 사용하여 출력 Tensor를 생성하는 forward를 정의합니다.

이 예제에서는 2-계층 신경망을 사용자 정의 Module의 서브클래스로 구현해보겠습니다.

In [19]:
import torch

In [23]:
class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        '''
        생성자에서 2개의 nn.Linear 모듈을 생성(Instantiate)하고, 멤버 변수로 지정합니다.
        '''
        super(TwoLayerNet, self).__init__() # 안 쓰면 AttributeError: cannot assign module before Module.__init__() call 에러 난다
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)
        
    def forward(self, x):
        '''
        순전파 함수에서는 입력 데이터의 Tensor를 받아서 출력 데이터의 Tensor를 반환해야
        합니다. Tensor 상의 임의의 연산자뿐만 아니라 생성자에서 정의한 모듈을 사용할 수
        있습니다.
        '''
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        return y_pred

In [21]:
# N은 배치 크기이며, D_in은 입력의 차원입니다;
# H는 은닉 계층의 차원이며, D_out은 출력 차원입니다.
N, D_in, H, D_out = 64, 1000, 100, 10

# 입력과 출력을 저장하기 위해 무작위 값을 갖는 Tensor를 생성합니다.
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

In [24]:
# 앞에서 정의한 클래스를 생성(Instantiating)해서 모델을 구성합니다.
model = TwoLayerNet(D_in, H, D_out)

In [25]:
# 손실함수와 Optimizer를 만듭니다. SGD 생성자에서 model.parameters()를 호출하면
# 모델의 멤버인 2개의 nnlinear 모듈의 학습 가능한 매개변수들이 포함됩니다.
criterion = torch.nn.MSELoss(size_average=False)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)

C:\ProgramData\Anaconda3\envs\keras_gpu\lib\site-packages\torch\nn\functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [26]:
for t in range(501):
    # 순전파 단계: 모델에 x를 전달하여 예상하는 y값을 계산합니다.
    y_pred = model(x)
    
    # 손실을 계산하고 출력합니다.
    loss = criterion(y_pred, y)
    if t%20 == 0:
        print(t, loss.item())
        
    # 변화도를 0으로 만들고, 역전파 단계를 수행하고, 가중치를 갱신합니다.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

0 628.2279663085938
20 189.5401153564453
40 59.99802780151367
60 19.44403648376465
80 7.085575580596924
100 2.8896589279174805
120 1.2778719663619995
140 0.5970858931541443
160 0.2892526686191559
180 0.14471444487571716
200 0.0740443617105484
220 0.03849857300519943
240 0.020270824432373047
260 0.010777910239994526
280 0.005776281468570232
300 0.0031193059403449297
320 0.0016958130290731788
340 0.0009269958827644587
360 0.000509349862113595
380 0.0002811844169627875
400 0.00015596796583849937
420 8.691451512277126e-05
440 4.865628216066398e-05
460 2.7355448764865287e-05
480 1.545016493764706e-05
500 8.764021913520992e-06


# PyTorch: 제어 흐름(Control Flow) + 가중치 공유(Weight Sharing)

동적 그래프와 가중치 공유의 예로, 매우 이상한 모델을 구현해보겠습니다. 각각의 순전파 단계에서 많은 은닉 계층을 갖는 완전히 연결(Fully-connected)된 ReLU 신경망이 무작위로 1~4 사이의 숫자를 선택하고, 동일한 가중치를 여러 번 재사용하여 가장 안쪽(innermost)에 있는 은닉 계층들을 게산합니다.

이 모델에서는 반복문을 구현하기 위해 일반적인 Python 제어 흐름을 사용하고, 순전파 단계를 정의할 때 단지 동일한 모듈을 여러번 재사용함으로써 내부(innermost) 게층들 간의 가중치 공유를 구현할 수 있습니다.

이 모델은 간단히 Module을 상속받는 서브클래스로 구현해보겠습니다.

In [18]:
import random
import torch

In [27]:
class DynamicNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        '''
        생성자에서는 순전파 단계에서 사용할 3개의 nn.Linear 개체(Instance)를 생성합니다.
        '''
        super(DynamicNet, self).__init__()
        self.input_linear = torch.nn.Linear(D_in, H)
        self.middle_linear = torch.nn.Linear(H, H)
        self.output_linear = torch.nn.Linear(H, D_out)
        
    def forward(self, x):
        '''
        모델의 순전파 단계에서, 무작위로 0,1,2 또는 3 중에 하나를 선택하고 
        은닉 계층 표현(representation)을 계산하기 위해 여러번 사용한 middle_linear
        모듈을 재사용합니다.
        
        각 순전파 단계에서 동적 연산 그래프를 구성하기 때문에, 모델의 순전파 단계를 
        정의할 때 반복문이나 조건문과 같이 일반적인 Python 제어 흐름 연산자를 사용할 
        수 있습니다.
        
        여기에서 연산 그래프를 정의할 때 동일한 모듈을 여러번 재사용하는 것이
        완벽하게 안전하다는 것을 알 수 있습니다. 이것이 각 모듈을 한 번만 사용하는
        Lua Torch보다 크게 개선된 부분입니다.
        '''
        h_relu = self.input_linear(x).clamp(min=0)
        for _ in range(random.randint(0,3)):
            h_relu = self.middle_linear(h_relu).clamp(min=0)
        y_pred = self.output_linear(h_relu)
        return y_pred       

In [28]:
# N은 배치 크기이며, D_in은 입력의 차원입니다;
# H는 은닉 계층의 차원이며, D_out은 출력 차원입니다.
N, D_in, H, D_out = 64, 1000, 100, 10

# 입력과 출력을 저장하기 위해 무작위 값을 갖는 Tensor를 생성합니다.
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

In [29]:
# 앞서 정의한 클래스를 생성(Instantiating)해서 모델을 구성합니다.
model = DynamicNet(D_in, H, D_out)

In [30]:
# 손실함수와 Optimizer를 만듭니다. 이 이상한 모델을 순수한 확률적 경사 하강법
# (Stochastic Gradient Descent)으로 학습하는 것은 어려우므로, momentum을 사용합니다.
criterion = torch.nn.MSELoss(size_average=False)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)

C:\ProgramData\Anaconda3\envs\keras_gpu\lib\site-packages\torch\nn\functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [31]:
for t in range(501):
    # 순전파 단계: 모델에 x를 전달하여 예상하는 y값을 계산합니다.
    y_pred = model(x)
    
    # 손실을 계산하고 출력합니다.
    loss = criterion(y_pred, y)
    if t%20 == 0:
        print(t, loss.item())
        
    # 변화도를 0으로 만들고, 역전파 단계를 수행하고, 가중치를 갱신합니다.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

0 637.1878662109375
20 489.172119140625
40 276.68145751953125
60 44.83914566040039
80 32.1054801940918
100 26.520095825195312
120 8.094974517822266
140 29.725610733032227
160 23.39359474182129
180 14.966519355773926
200 1.7262383699417114
220 2.004809617996216
240 1.0725444555282593
260 0.5610405802726746
280 4.366574287414551
300 1.9497673511505127
320 3.626225471496582
340 0.7234248518943787
360 0.9468593597412109
380 1.7481191158294678
400 0.616844654083252
420 1.9778934717178345
440 0.10392443090677261
460 0.6005434989929199
480 0.957409143447876
500 0.10011167824268341
